# Toy GCN Model Ample Demo

In [8]:
import sys
sys.path.append("/home/aw1223/ip/agile/")

from sdk.ample import Ample

from sdk.graphs.random_graph import RandomGraph

#TODO change to just models in file sturcture
from sdk.models.models import GCN_Model, GAT_Model, GraphSAGE_Model, GIN_Model, GCN_MLP_Model, MLP_Model, Edge_Embedding_Model, Interaction_Net_Model

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
ample = Ample()
edge = True

#TODO fix this : If model does not use edges, dont set edges to be true - will brrak things 
graph = RandomGraph(num_nodes=10, avg_degree=1, num_channels=32, graph_precision="FLOAT_32",edge_dim=32,edges = edge) #TODO add var


In [10]:
#Trace model, find the constituent models e.g GCN, Interaction net and create a data flow graph

model_cls = MLP_Model

model = model_cls()


# tracer = GraphTracer(model)
# tracer.get_input_output_layers()
# # tracer.print_input_output_layers()
# tracer.draw()




In [4]:
#Use initalize to construct memory map for each model and then string together the dataflow between models
#For each mode, init mem and store to seperate files?
ample.initialize_memory(model,graph)



2it [00:00, 12318.07it/s]


In [5]:
# ample.sim()



In [6]:
#Program Agile using device
# ample.copy_data_to_device(data_file_name)
# ample.execute()
# ample.retrieve_data()

## Neural-LAM

In [12]:
# Standard library
import random
import time
from argparse import ArgumentParser

# Third-party
import pytorch_lightning as pl
import torch
from lightning_fabric.utilities import seed

# First-party
from neural_lam import constants, utils
from neural_lam.models.graph_lam import GraphLAM
from neural_lam.models.hi_lam import HiLAM
from neural_lam.models.hi_lam_parallel import HiLAMParallel
from neural_lam.weather_dataset import WeatherDataset


import argparse

# Manually create the args Namespace object with the desired parameters
args = argparse.Namespace(
    dataset='meps_example',
    model='graph_lam',
    subset_ds=0,
    seed=42,
    n_workers=4,
    epochs=200,
    batch_size=1,
    load=None,
    restore_opt=0,
    precision=32,
    graph='1level',
    hidden_dim=32,
    hidden_layers=1,
    processor_layers=1,
    mesh_aggr='sum',
    output_std=0,
    ar_steps=1,
    control_only=0,
    loss='wmse',
    step_length=3,
    lr=0.001,
    val_interval=1,
    eval='test',
    n_example_pred=1
)

print('args', args)
print('args', args.dataset)

args Namespace(dataset='meps_example', model='graph_lam', subset_ds=0, seed=42, n_workers=4, epochs=200, batch_size=1, load=None, restore_opt=0, precision=32, graph='1level', hidden_dim=32, hidden_layers=1, processor_layers=1, mesh_aggr='sum', output_std=0, ar_steps=1, control_only=0, loss='wmse', step_length=3, lr=0.001, val_interval=1, eval='test', n_example_pred=1)
args meps_example


In [13]:

MODELS = {
    "graph_lam": GraphLAM,
    "hi_lam": HiLAM,
    "hi_lam_parallel": HiLAMParallel,
}


# Asserts for arguments
assert args.model in MODELS, f"Unknown model: {args.model}"
assert args.step_length <= 3, "Too high step length"
assert args.eval in (
    None,
    "val",
    "test",
), f"Unknown eval setting: {args.eval}"
print('args',args)
# Get an (actual) random run id as a unique identifier
random_run_id = random.randint(0, 9999)

# Set seed
seed.seed_everything(args.seed)

# Load data
train_loader = torch.utils.data.DataLoader(
    WeatherDataset(
        args.dataset,
        pred_length=args.ar_steps,
        split="train",
        subsample_step=args.step_length,
        subset=bool(args.subset_ds),
        control_only=args.control_only,
    ),
    args.batch_size,
    shuffle=True,
    num_workers=args.n_workers,
)
max_pred_length = (65 // args.step_length) - 2  # 19
val_loader = torch.utils.data.DataLoader(
    WeatherDataset(
        args.dataset,
        pred_length=max_pred_length,
        split="val",
        subsample_step=args.step_length,
        subset=bool(args.subset_ds),
        control_only=args.control_only,
    ),
    args.batch_size,
    shuffle=False,
    num_workers=args.n_workers,
)

# Instantiate model + trainer
if torch.cuda.is_available():
    device_name = "cuda"
    torch.set_float32_matmul_precision(
        "high"
    )  # Allows using Tensor Cores on A100s
else:
    device_name = "cpu"

# Load model parameters Use new args for model
model_class = MODELS[args.model]
# if args.load:
#     model = model_class.load_from_checkpoint(args.load, args=args)
#     if args.restore_opt:
#         # Save for later
#         # Unclear if this works for multi-GPU
#         model.opt_state = torch.load(args.load)["optimizer_states"][0]
# else:
#     model = model_class(args)

prefix = "subset-" if args.subset_ds else ""
if args.eval:
    prefix = prefix + f"eval-{args.eval}-"
run_name = (
    f"{prefix}{args.model}-{args.processor_layers}x{args.hidden_dim}-"
    f"{time.strftime('%m_%d_%H')}-{random_run_id:04d}"
)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=f"saved_models/{run_name}",
    filename="min_val_loss",
    monitor="val_mean_loss",
    mode="min",
    save_last=True,
)
logger = pl.loggers.WandbLogger(
    project=constants.WANDB_PROJECT, name=run_name, config=args
)
trainer = pl.Trainer(
    max_epochs=args.epochs,
    deterministic=True,
    strategy="ddp_notebook",
    accelerator=device_name,
    logger=logger,
    log_every_n_steps=1,
    callbacks=[checkpoint_callback],
    check_val_every_n_epoch=args.val_interval,
    precision=args.precision,
)

# Only init once, on rank 0 only
if trainer.global_rank == 0:
    utils.init_wandb_metrics(logger)  # Do after wandb.init

if args.eval:
    if args.eval == "val":
        eval_loader = val_loader
    else:  # Test
        eval_loader = torch.utils.data.DataLoader(
            WeatherDataset(
                args.dataset,
                pred_length=max_pred_length,
                split="test",
                subsample_step=args.step_length,
                subset=bool(args.subset_ds),
            ),
            args.batch_size,
            shuffle=False,
            num_workers=args.n_workers,
        )

    print(f"Running evaluation on {args.eval}")
    # trainer.test(model=model, dataloaders=eval_loader)
else:
    # Train model
    trainer.fit(
        model=model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader,
    )


Seed set to 42


args Namespace(dataset='meps_example', model='graph_lam', subset_ds=0, seed=42, n_workers=4, epochs=200, batch_size=1, load=None, restore_opt=0, precision=32, graph='1level', hidden_dim=32, hidden_layers=1, processor_layers=1, mesh_aggr='sum', output_std=0, ar_steps=1, control_only=0, loss='wmse', step_length=3, lr=0.001, val_interval=1, eval='test', n_example_pred=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id 3h67z6h7.


Running evaluation on test


In [27]:
import torch
from torch import nn
import pytorch_lightning as pl

def trace_lightning_model_io_with_loader(model, dataloader):
    """
    Traces the PyTorch Lightning model to get the names of input and output tensors 
    along with the module hierarchy using a provided DataLoader.

    Args:
        model (pl.LightningModule): The PyTorch Lightning model to trace.
        dataloader (torch.utils.data.DataLoader): The DataLoader to supply inputs to the model.

    Returns:
        dict: A dictionary where keys are module names and values are tuples of (input_tensor_names, output_tensor_names).
    """
    io_dict = {}
    tensor_id_to_name = {}
    
    def register_hooks(module, module_name):
        def hook(module, inputs, outputs):
            # Record the inputs
            input_names = []
            for i, inp in enumerate(inputs):
                if isinstance(inp, torch.Tensor):
                    tensor_id = id(inp)
                    if tensor_id not in tensor_id_to_name:
                        tensor_name = f"{module_name}_input_{i}"
                        tensor_id_to_name[tensor_id] = tensor_name
                    else:
                        tensor_name = tensor_id_to_name[tensor_id]
                    input_names.append(tensor_name)

            # Record the outputs
            output_names = []
            if isinstance(outputs, (tuple, list)):
                for i, out in enumerate(outputs):
                    if isinstance(out, torch.Tensor):
                        tensor_id = id(out)
                        tensor_name = f"{module_name}_output_{i}"
                        tensor_id_to_name[tensor_id] = tensor_name
                        output_names.append(tensor_name)
            else:
                if isinstance(outputs, torch.Tensor):
                    tensor_id = id(outputs)
                    tensor_name = f"{module_name}_output_0"
                    tensor_id_to_name[tensor_id] = tensor_name
                    output_names.append(tensor_name)

            # Store the mapping for this module
            io_dict[module_name] = (input_names, output_names)
        
        module.register_forward_hook(hook)

    # Register hooks to all modules
    for name, module in model.named_modules():
        register_hooks(module, name)

    # Perform a forward pass using the dataloader to trigger the hooks
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            model.common_step(batch)  # Trigger forward pass
            break  # Only need one batch to trace

    return io_dict

In [29]:

hidden_dim = 32
hidden_layers = 1
mesh_aggr = args.mesh_aggr
lr = args.lr
dataset = args.dataset
output_std = args.output_std
loss = args.loss
step_length = args.step_length
n_example_pred = args.n_example_pred
graph = args.graph
processor_layers = args.processor_layers
model = GraphLAM(hidden_dim, hidden_layers, mesh_aggr,lr, dataset, output_std, loss, step_length, n_example_pred,graph,processor_layers)

model_io = trace_lightning_model_io_with_loader(model, eval_loader)

# Print the module I/O information
for module_name, (input_names, output_names) in model_io.items():
    print(f"Module: {module_name}")
    print(f"  Input Tensors: {input_names}")
    print(f"  Output Tensors: {output_names}")

Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
Edges in subgraphs: m2m=51520, g2m=100656, m2g=255136
Module: grid_embedder.0
  Input Tensors: ['g2m_gnn_output_0']
  Output Tensors: ['grid_embedder.0_output_0']
Module: grid_embedder.1
  Input Tensors: ['grid_embedder.0_output_0']
  Output Tensors: ['grid_embedder.1_output_0']
Module: grid_embedder.2
  Input Tensors: ['grid_embedder.1_output_0']
  Output Tensors: ['grid_embedder.2_output_0']
Module: grid_embedder.3
  Input Tensors: ['grid_embedder.2_output_0']
  Output Tensors: ['grid_embedder.3_output_0']
Module: grid_embedder
  Input Tensors: ['g2m_gnn_output_0']
  Output Tensors: ['grid_embedder_output_0']
Module: g2m_embedder.0
  Input Tensors: ['g2m_embedder.0_input_0']
  Output Tensors: ['g2m_embedder.0_output_0']
Module: g2m_embedder.1
  Input Tensors: ['g2m_embedder.0_output_0']
  Output Tensors: ['g2m_embedder.1_output_0']
Module: g2m_embedder.2
  Input Tensors: ['g2m_embedder.1_output_0']
  Output Tensors: ['g2m_embedder

In [30]:
from graphviz import Digraph

def plot_model_io_graph(io_dict, output_file='model_io_graph'):
    """
    Plots the model's input/output tensor flow using graphviz.

    Args:
        io_dict (dict): A dictionary where keys are module names and values are tuples of 
                        (input_tensor_names, output_tensor_names).
        output_file (str): The name of the output file (without extension).

    Returns:
        Digraph: The generated graph object.
    """
    dot = Digraph(comment='Model I/O Graph')
    dot.attr(rankdir='LR', size='10')

    for module_name, (input_names, output_names) in io_dict.items():
        # Create node for the module
        dot.node(module_name, module_name, shape='box')

        # Create nodes for inputs
        for input_name in input_names:
            if input_name not in dot.node_attr:
                dot.node(input_name, input_name, shape='ellipse')
            dot.edge(input_name, module_name)

        # Create nodes for outputs
        for output_name in output_names:
            if output_name not in dot.node_attr:
                dot.node(output_name, output_name, shape='ellipse')
            dot.edge(module_name, output_name)

    # Save the graph to a file and render it
    dot.render(output_file, format='png')

    return dot

# Example usage:
# Assuming model_io is the I/O dictionary generated from the previous step

graph = plot_model_io_graph(model_io, output_file='model_io_graph')

In [34]:
from graphviz import Digraph

def plot_model_io_graph(io_dict, output_file='model_io_graph', format='svg', dpi=300):
    """
    Plots the model's input/output tensor flow using graphviz.

    Args:
        io_dict (dict): A dictionary where keys are module names and values are tuples of 
                        (input_tensor_names, output_tensor_names).
        output_file (str): The name of the output file (without extension).
        format (str): The format of the output file (e.g., 'png', 'svg', 'pdf').
        dpi (int): The resolution in dots per inch (DPI) for the output image.

    Returns:
        Digraph: The generated graph object.
    """
    dot = Digraph(comment='Model I/O Graph')
    dot.attr(rankdir='LR', size='10')

    for module_name, (input_names, output_names) in io_dict.items():
        # Create node for the module
        dot.node(module_name, module_name, shape='box')

        # Create nodes for inputs
        for input_name in input_names:
            if input_name not in dot.node_attr:
                dot.node(input_name, input_name, shape='ellipse')
            dot.edge(input_name, module_name)

        # Create nodes for outputs
        for output_name in output_names:
            if output_name not in dot.node_attr:
                dot.node(output_name, output_name, shape='ellipse')
            dot.edge(module_name, output_name)

    # Set output resolution
    dot.attr(dpi=str(dpi))

    # Save the graph to a file and render it
    dot.render(output_file, format=format)

    return dot

# Example usage:
# Assuming model_io is the I/O dictionary generated from the previous step

graph = plot_model_io_graph(model_io, output_file='model_io_graph', format='png', dpi=1000)

In [33]:
from graphviz import Digraph

def plot_simplified_model_io_graph(io_dict, output_file='simplified_model_io_graph', format='png', dpi=300):
    """
    Plots a simplified version of the model's input/output tensor flow using graphviz.
    
    Args:
        io_dict (dict): A dictionary where keys are module names and values are tuples of 
                        (input_tensor_names, output_tensor_names).
        output_file (str): The name of the output file (without extension).
        format (str): The format of the output file (e.g., 'png', 'svg', 'pdf').
        dpi (int): The resolution in dots per inch (DPI) for the output image.
    
    Returns:
        Digraph: The generated graph object.
    """
    dot = Digraph(comment='Simplified Model I/O Graph')
    dot.attr(rankdir='TB', size='10')  # TB for top-bottom layout, LR for left-right

    # Track unique tensors and simplify connections
    tensor_seen = set()

    for module_name, (input_names, output_names) in io_dict.items():
        # Create a node for the module
        dot.node(module_name, module_name, shape='box')

        # Only show unique input tensors to avoid clutter
        for input_name in input_names:
            if input_name not in tensor_seen:
                dot.node(input_name, input_name, shape='ellipse')
                tensor_seen.add(input_name)
            dot.edge(input_name, module_name)

        # Only show unique output tensors to avoid clutter
        for output_name in output_names:
            if output_name not in tensor_seen:
                dot.node(output_name, output_name, shape='ellipse')
                tensor_seen.add(output_name)
            dot.edge(module_name, output_name)

    # Set output resolution
    dot.attr(dpi=str(dpi))

    # Save the graph to a file and render it
    dot.render(output_file, format=format)

    return dot

# Example usage:
# Assuming model_io is the I/O dictionary generated from the previous step

graph = plot_simplified_model_io_graph(model_io, output_file='simplified_model_io_graph', format='png', dpi=1000)

# Final

In [35]:
import torch
from graphviz import Digraph

def plot_simplified_model_io_graph(io_dict, output_file='simplified_model_io_graph', format='png', dpi=300):
    """
    Plots a simplified version of the model's input/output tensor flow using graphviz.
    
    Args:
        io_dict (dict): A dictionary where keys are module names and values are tuples of 
                        (input_tensor_names, output_tensor_names).
        output_file (str): The name of the output file (without extension).
        format (str): The format of the output file (e.g., 'png', 'svg', 'pdf').
        dpi (int): The resolution in dots per inch (DPI) for the output image.
    
    Returns:
        Digraph: The generated graph object.
    """
    dot = Digraph(comment='Simplified Model I/O Graph')
    dot.attr(rankdir='TB', size='10')  # TB for top-bottom layout

    # Track unique tensors and simplify connections
    tensor_seen = set()

    for module_name, (input_names, output_names) in io_dict.items():
        # Extract the top-level module name only
        top_level_module_name = module_name.split('.')[0]

        # Create a node for the top-level module
        if top_level_module_name not in dot.node_attr:
            dot.node(top_level_module_name, top_level_module_name, shape='box')

        # Only show unique input tensors to avoid clutter
        for input_name in input_names:
            if input_name not in tensor_seen:
                tensor_seen.add(input_name)
                dot.node(input_name, input_name, shape='ellipse')
            dot.edge(input_name, top_level_module_name)

        # Only show unique output tensors to avoid clutter
        for output_name in output_names:
            if output_name not in tensor_seen:
                tensor_seen.add(output_name)
                dot.node(output_name, output_name, shape='ellipse')
            dot.edge(top_level_module_name, output_name)

    # Set output resolution
    dot.attr(dpi=str(dpi))

    # Save the graph to a file and render it
    dot.render(output_file, format=format)

    return dot

def trace_lightning_model_io_with_loader(model, dataloader):
    """
    Traces the PyTorch Lightning model to get the names of input and output tensors 
    along with the module hierarchy using a provided DataLoader.

    Args:
        model (pl.LightningModule): The PyTorch Lightning model to trace.
        dataloader (torch.utils.data.DataLoader): The DataLoader to supply inputs to the model.

    Returns:
        dict: A dictionary where keys are module names and values are tuples of (input_tensor_names, output_tensor_names).
    """
    io_dict = {}
    tensor_id_to_name = {}
    
    def register_hooks(module, module_name):
        def hook(module, inputs, outputs):
            # Capture the top-level module name
            top_level_module_name = module_name.split('.')[0]
            # Record the inputs
            input_names = []
            for i, inp in enumerate(inputs):
                if isinstance(inp, torch.Tensor):
                    tensor_id = id(inp)
                    if tensor_id not in tensor_id_to_name:
                        tensor_name = f"{top_level_module_name}_input_{i}"
                        tensor_id_to_name[tensor_id] = tensor_name
                    else:
                        tensor_name = tensor_id_to_name[tensor_id]
                    input_names.append(tensor_name)

            # Record the outputs
            output_names = []
            if isinstance(outputs, (tuple, list)):
                for i, out in enumerate(outputs):
                    if isinstance(out, torch.Tensor):
                        tensor_id = id(out)
                        tensor_name = f"{top_level_module_name}_output_{i}"
                        tensor_id_to_name[tensor_id] = tensor_name
                        output_names.append(tensor_name)
            else:
                if isinstance(outputs, torch.Tensor):
                    tensor_id = id(outputs)
                    tensor_name = f"{top_level_module_name}_output_0"
                    tensor_id_to_name[tensor_id] = tensor_name
                    output_names.append(tensor_name)

            # Store the mapping for this module
            io_dict[top_level_module_name] = (input_names, output_names)
        
        module.register_forward_hook(hook)

    # Register hooks to top-level modules only
    for name, module in model.named_children():
        register_hooks(module, name)

    # Perform a forward pass using the dataloader to trigger the hooks
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            model.common_step(batch)  # Trigger forward pass
            break  # Only need one batch to trace

    return io_dict



hidden_dim = 32
hidden_layers = 1
mesh_aggr = args.mesh_aggr
lr = args.lr
dataset = args.dataset
output_std = args.output_std
loss = args.loss
step_length = args.step_length
n_example_pred = args.n_example_pred
graph = args.graph
processor_layers = args.processor_layers
model = GraphLAM(hidden_dim, hidden_layers, mesh_aggr,lr, dataset, output_std, loss, step_length, n_example_pred,graph,processor_layers)



model = GraphLAM(hidden_dim, hidden_layers, mesh_aggr, lr, dataset, output_std, loss, step_length, n_example_pred, graph, processor_layers)

# Assuming eval_loader is your DataLoader
# eval_loader = torch.utils.data.DataLoader(...)  # Set up your DataLoader

# Trace the model I/O
model_io = trace_lightning_model_io_with_loader(model, eval_loader)

# Plot the simplified model I/O graph
graph = plot_simplified_model_io_graph(model_io, output_file='simplified_model_io_graph', format='png', dpi=300)

# Print the module I/O information in the desired format
for module_name, (input_names, output_names) in model_io.items():
    print(f"Module: {module_name}")
    for input_name in input_names:
        print(f"  Input Tensor: {input_name}")
    for output_name in output_names:
        print(f"  Output Tensor: {output_name}")

Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
Edges in subgraphs: m2m=51520, g2m=100656, m2g=255136
Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
Edges in subgraphs: m2m=51520, g2m=100656, m2g=255136
Module: grid_embedder
  Input Tensor: g2m_gnn_output_0
  Output Tensor: grid_embedder_output_0
Module: g2m_embedder
  Input Tensor: g2m_embedder_input_0
  Output Tensor: g2m_embedder_output_0
Module: m2g_embedder
  Input Tensor: m2g_embedder_input_0
  Output Tensor: m2g_embedder_output_0
Module: mesh_embedder
  Input Tensor: mesh_embedder_input_0
  Output Tensor: mesh_embedder_output_0
Module: g2m_gnn
  Input Tensor: grid_embedder_output_0
  Input Tensor: grid_embedder_output_0
  Input Tensor: output_map_output_0
  Output Tensor: g2m_gnn_output_0
Module: encoding_grid_mlp
  Input Tensor: grid_embedder_output_0
  Output Tensor: encoding_grid_mlp_output_0
Module: m2m_embedder
  Input Tensor: m2m_embedder_input_0
  Output Tensor: m2m_embedder_output_0
Module: processor
  Input

In [ ]:
import sys
sys.path.append("/home/aw1223/ip/agile/")

from sdk.ample import Ample


from sdk.graph_tracer import ManualGraphInspector
tracer = ManualGraphInspector(model)
# tracer.get_input_output_layers()
# # tracer.print_input_output_layers()
# tracer.draw()


In [ ]:
# To inspect the whole model structure and its modules
for name, module in model.named_modules():
    print(name, module)

 MLP_Model(
  (layers): ModuleList(
    (0): Linear(in_features=32, out_features=32, bias=True)
  )
)
layers ModuleList(
  (0): Linear(in_features=32, out_features=32, bias=True)
)
layers.0 Linear(in_features=32, out_features=32, bias=True)


In [ ]:
import torch
from torchviz import make_dot

# Example of using minimal data for tracing
# You can use a small tensor with minimal dimensions if you want to keep it simple
minimal_input = torch.randn(1, 1, 1)  # Adjust the dimensions to match what your model expects

# Trace the model
traced_model = torch.jit.trace(model, minimal_input)

# Generate the visualization
make_dot(traced_model.graph, params=dict(model.named_parameters())).render("model_graph", format="png")

TypeError: MLP_Model.forward() missing 1 required positional argument: 'edge_index'

In [ ]:
import torch
import torch.fx

# Assuming GraphLAM is defined as shown before
# and has a proper forward method

# Instantiate your model
hidden_dim = args.hidden_dim
hidden_layers = args.hidden_layers
mesh_aggr = args.mesh_aggr
lr = args.lr
dataset = args.dataset
output_std = args.output_std
loss = args.loss
step_length = args.step_length
n_example_pred = args.n_example_pred
graph = args.graph
processor_layers = args.processor_layers
model = GraphLAM(hidden_dim, hidden_layers, mesh_aggr,lr, dataset, output_std, loss, step_length, n_example_pred,graph,processor_layers)

# Create an input that matches the expected input shape
dummy_input = torch.randn(1, 1, 1)  # Adjust to match your model's input shape

# Use torch.fx to trace the model
# traced_model = torch.fx.symbolic_trace(model)
traced_model = torch.fx.symbolic_trace(model)
# Print the graph representation
# print(traced_model.graph)

# Optionally visualize the graph (requires graphviz)
from torch.fx import GraphModule
from torch.fx.graph import Graph
from torch.fx.graph_module import GraphModule

def visualize_fx_graph(traced: GraphModule, input_shape: torch.Size):
    dummy_input = torch.randn(input_shape)
    dot = make_dot(traced(dummy_input), params=dict(traced.named_parameters()))
    dot.render("traced_model_graph", format="png")

# Visualize the traced graph (provide the correct input shape)
visualize_fx_graph(traced_model, input_shape=(1, 1, 1))  # Adjust input shape

get_num_mesh
Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
input_dim 32
hidden_dim 32
hidden_layers 1
input_dim 32
hidden_dim 32
hidden_layers 1
Edges in subgraphs: m2m=51520, g2m=100656, m2g=255136
input_dim 32
hidden_dim 32
hidden_layers 1
stage1
stage2


TraceError: Proxy object cannot be iterated. This can be attempted when the Proxy is used in a loop or as a *args or **kwargs function argument. See the torch.fx docs on pytorch.org for a more detailed explanation of what types of control flow can be traced, and check out the Proxy docstring for help troubleshooting Proxy iteration errors

In [ ]:
import torch
import torch.fx as fx

class CustomTracer(fx.Tracer):
    def is_leaf_module(self, m: torch.nn.Module, module_qualified_name: str) -> bool:
        # Treat InteractionNet as a leaf module (don't trace into it)
        if isinstance(m, InteractionNet):
            return True
        return super().is_leaf_module(m, module_qualified_name)

# Trace using the custom tracer
tracer = CustomTracer()
traced_model = tracer.trace(model)

# Create a GraphModule from the traced graph
graph_module = fx.GraphModule(model, traced_model)

# Print or visualize the traced graph
print(graph_module.graph)

TraceError: Proxy object cannot be iterated. This can be attempted when the Proxy is used in a loop or as a *args or **kwargs function argument. See the torch.fx docs on pytorch.org for a more detailed explanation of what types of control flow can be traced, and check out the Proxy docstring for help troubleshooting Proxy iteration errors

In [ ]:
import torch
import torch_geometric as pyg
import torch.utils.data
import torch.fx as fx
import pytorch_lightning as pl
from torchviz import make_dot

# Setting seed for reproducibility
torch.manual_seed(args.seed)



hidden_dim = args.hidden_dim
hidden_layers = args.hidden_layers
mesh_aggr = args.mesh_aggr
lr = args.lr
dataset = args.dataset
output_std = args.output_std
loss = args.loss
step_length = args.step_length
n_example_pred = args.n_example_pred
graph = args.graph
processor_layers = args.processor_layers
# model = GraphLAM(hidden_dim, hidden_layers, mesh_aggr,lr, dataset, output_std, loss, step_length, n_example_pred,graph,processor_layers)

# Instantiate model
model_class = MODELS[args.model]
model = model_class(hidden_dim, hidden_layers, mesh_aggr,lr, dataset, output_std, loss, step_length, n_example_pred,graph,processor_layers)

# Create dummy inputs for visualization
# This will depend on the input requirements of the model, adjust accordingly
send_rep = torch.randn((2, 10, 16))  # Example tensor, adjust dimensions based on model requirements
rec_rep = torch.randn((2, 10, 16))   # Example tensor, adjust dimensions based on model requirements
edge_rep = torch.randn((2, 10, 16))  # Example tensor, adjust dimensions based on model requirements

# Forward pass through the model
output = model(send_rep, rec_rep, edge_rep)

# Visualize the computational graph using torchviz
dot = make_dot(output, params=dict(model.named_parameters()))
dot.render("model_graph", format="png")

# If torch.fx is preferred for tracing
traced_model = fx.symbolic_trace(model)
print(traced_model.graph)

get_num_mesh
Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
input_dim 32
hidden_dim 32
hidden_layers 1
input_dim 32
hidden_dim 32
hidden_layers 1
Edges in subgraphs: m2m=51520, g2m=100656, m2g=255136
input_dim 32
hidden_dim 32
hidden_layers 1
stage1
stage2


NotImplementedError: Module [GraphLAM] is missing the required "forward" function

In [ ]:
def trace_model_connections(model):
    def recurse(obj, depth=0):
        indent = "    " * depth
        if hasattr(obj, 'modules'):
            for name, module in obj.named_children():
                if list(module.children()):  # Check if module has children
                    print(f"{indent}{name} ({module.__class__.__name__})")
                    recurse(module, depth + 1)
                else:
                    print(f"{indent}{name} ({module.__class__.__name__})")
        elif hasattr(obj, '__call__'):
            print(f"{indent}{obj.__class__.__name__}")

    print(f"Model: {model.__class__.__name__}")
    recurse(model)

# Assuming `model` is instantiated and provided as an instance of a class like GraphLAM
trace_model_connections(model)

Model: GraphLAM
grid_embedder (Sequential)
    0 (Linear)
    1 (SiLU)
    2 (Linear)
    3 (LayerNorm)
g2m_embedder (Sequential)
    0 (Linear)
    1 (SiLU)
    2 (Linear)
    3 (LayerNorm)
m2g_embedder (Sequential)
    0 (Linear)
    1 (SiLU)
    2 (Linear)
    3 (LayerNorm)
g2m_gnn (InteractionNetWrapper)
    interaction_net (InteractionNet)
        aggr_module (SumAggregation)
        edge_mlp (Sequential)
            0 (Linear)
            1 (SiLU)
            2 (Linear)
            3 (LayerNorm)
        aggr_mlp (Sequential)
            0 (Linear)
            1 (SiLU)
            2 (Linear)
            3 (LayerNorm)
encoding_grid_mlp (Sequential)
    0 (Linear)
    1 (SiLU)
    2 (Linear)
    3 (LayerNorm)
m2g_gnn (InteractionNetWrapper)
    interaction_net (InteractionNet)
        aggr_module (SumAggregation)
        edge_mlp (Sequential)
            0 (Linear)
            1 (SiLU)
            2 (Linear)
            3 (LayerNorm)
        aggr_mlp (Sequential)
            0 (Line

In [ ]:
import torch
from collections import defaultdict

def trace_model_connections(model):
    connections = defaultdict(list)

    def recurse(obj, depth=0, parent=None):
        for name, module in obj.named_children():
            current_module_name = f"{parent}.{name}" if parent else name

            if list(module.children()):  # If the module has children, recurse into them
                recurse(module, depth + 1, current_module_name)
            else:
                connections[parent].append(current_module_name)

    recurse(model)
    return dict(connections)

# Assuming `model` is an instance of your model, such as GraphLAM
connections_dict = trace_model_connections(model)

# Print the connections dictionary
for module, inputs in connections_dict.items():
    print(f"{module}: {inputs}")

[autoreload of numpy.core.multiarray failed: Traceback (most recent call last):
  File "/home/aw1223/anaconda3/envs/ample/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/aw1223/anaconda3/envs/ample/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/aw1223/anaconda3/envs/ample/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/aw1223/anaconda3/envs/ample/lib/python3.11/site-packages/numpy/core/multiarray.py", line 1, in <module>
    from numpy._core import multiarray
  File "/home/aw1223/anaconda3/envs/ample/lib/python3.11/s

grid_embedder: ['grid_embedder.0', 'grid_embedder.1', 'grid_embedder.2', 'grid_embedder.3']
g2m_embedder: ['g2m_embedder.0', 'g2m_embedder.1', 'g2m_embedder.2', 'g2m_embedder.3']
m2g_embedder: ['m2g_embedder.0', 'm2g_embedder.1', 'm2g_embedder.2', 'm2g_embedder.3']
g2m_gnn.interaction_net: ['g2m_gnn.interaction_net.aggr_module']
g2m_gnn.interaction_net.edge_mlp: ['g2m_gnn.interaction_net.edge_mlp.0', 'g2m_gnn.interaction_net.edge_mlp.1', 'g2m_gnn.interaction_net.edge_mlp.2', 'g2m_gnn.interaction_net.edge_mlp.3']
g2m_gnn.interaction_net.aggr_mlp: ['g2m_gnn.interaction_net.aggr_mlp.0', 'g2m_gnn.interaction_net.aggr_mlp.1', 'g2m_gnn.interaction_net.aggr_mlp.2', 'g2m_gnn.interaction_net.aggr_mlp.3']
encoding_grid_mlp: ['encoding_grid_mlp.0', 'encoding_grid_mlp.1', 'encoding_grid_mlp.2', 'encoding_grid_mlp.3']
m2g_gnn.interaction_net: ['m2g_gnn.interaction_net.aggr_module']
m2g_gnn.interaction_net.edge_mlp: ['m2g_gnn.interaction_net.edge_mlp.0', 'm2g_gnn.interaction_net.edge_mlp.1', 'm2g_gn

In [ ]:
from collections import defaultdict

def trace_model_connections_via_structure(model):
    connections = defaultdict(list)
    module_names = list(model.named_modules())
    
    for i, (name, module) in enumerate(module_names):
        if i > 0:  # Skip the first one as it's the entire model
            # Get the parent module
            parent_name, parent_module = module_names[i-1]
            # Only consider leaf modules (modules without children)
            if len(list(module.children())) == 0:
                connections[parent_name].append(name)
                
    return dict(connections)

# Assuming `model` is your instantiated model, e.g., GraphLAM
connections_dict = trace_model_connections_via_structure(model)

# Print the connections dictionary
for module, connections in connections_dict.items():
    print(f"{module}: {connections}")

grid_embedder: ['grid_embedder.0']
grid_embedder.0: ['grid_embedder.1']
grid_embedder.1: ['grid_embedder.2']
grid_embedder.2: ['grid_embedder.3']
g2m_embedder: ['g2m_embedder.0']
g2m_embedder.0: ['g2m_embedder.1']
g2m_embedder.1: ['g2m_embedder.2']
g2m_embedder.2: ['g2m_embedder.3']
m2g_embedder: ['m2g_embedder.0']
m2g_embedder.0: ['m2g_embedder.1']
m2g_embedder.1: ['m2g_embedder.2']
m2g_embedder.2: ['m2g_embedder.3']
g2m_gnn.interaction_net: ['g2m_gnn.interaction_net.aggr_module']
g2m_gnn.interaction_net.edge_mlp: ['g2m_gnn.interaction_net.edge_mlp.0']
g2m_gnn.interaction_net.edge_mlp.0: ['g2m_gnn.interaction_net.edge_mlp.1']
g2m_gnn.interaction_net.edge_mlp.1: ['g2m_gnn.interaction_net.edge_mlp.2']
g2m_gnn.interaction_net.edge_mlp.2: ['g2m_gnn.interaction_net.edge_mlp.3']
g2m_gnn.interaction_net.aggr_mlp: ['g2m_gnn.interaction_net.aggr_mlp.0']
g2m_gnn.interaction_net.aggr_mlp.0: ['g2m_gnn.interaction_net.aggr_mlp.1']
g2m_gnn.interaction_net.aggr_mlp.1: ['g2m_gnn.interaction_net.aggr_

In [ ]:
import torch
import inspect
from collections import defaultdict
from neural_lam.interaction_net import InteractionNet
 
def trace_model_hierarchy(model):
    """
    Traces the given model to extract and print the interaction nets, embedding modules,
    and their hierarchical/computational relationships.
    
    Args:
        model (torch.nn.Module): The model to trace.
    
    Returns:
        hierarchy (dict): A nested dictionary representing the model's hierarchy.
    """
    hierarchy = defaultdict(dict)
    
    def trace_module(module, parent_name=""):
        """
        Recursively traces the modules and stores information about
        Interaction Nets, Embedding Modules, and the hierarchy.
        """
        # Name of the current module
        module_name = parent_name + ('.' if parent_name else '') + type(module).__name__
        
        # Check if the module is an InteractionNetWrapper or InteractionNet
        if isinstance(module, (InteractionNet)):
            hierarchy['interaction_nets'][module_name] = module
        
        # Check if the module is an embedding module (MLP in this case)
        if hasattr(module, 'mlp_blueprint_end'):
            hierarchy['embedding_modules'][module_name] = module
        
        # Recursively trace child modules
        for name, child in module.named_children():
            trace_module(child, parent_name=module_name)
    
    # Start tracing from the given model
    trace_module(model)
    
    # Function to print the hierarchy in a readable format
    def print_hierarchy(hierarchy, indent=0):
        """
        Recursively prints the model hierarchy.
        """
        for key, value in hierarchy.items():
            if isinstance(value, dict):
                print(" " * indent + f"{key}:")
                print_hierarchy(value, indent + 4)
            else:
                print(" " * indent + f"{key}: {type(value).__name__}")

    print("Model Hierarchy:")
    print_hierarchy(hierarchy)
    
    return hierarchy

hidden_dim = args.hidden_dim
hidden_layers = args.hidden_layers
mesh_aggr = args.mesh_aggr
lr = args.lr
dataset = args.dataset
output_std = args.output_std
loss = args.loss
step_length = args.step_length
n_example_pred = args.n_example_pred
graph = args.graph
processor_layers = args.processor_layers
model = GraphLAM(hidden_dim, hidden_layers, mesh_aggr,lr, dataset, output_std, loss, step_length, n_example_pred,graph,processor_layers)

# Cre
# For BaseHiGraphModel (assuming required arguments are passed)
# args = ...  # Populate with required arguments for BaseHiGraphModel
# base_hi_graph_model = BaseHiGraphModel(args)

# Trace the models
graph_lam_hierarchy = trace_model_hierarchy(model)
# base_hi_graph_hierarchy = trace_model_hierarchy(base_hi_graph_model)

get_num_mesh
Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
Edges in subgraphs: m2m=51520, g2m=100656, m2g=255136
stage1
stage2
Model Hierarchy:
embedding_modules:
    GraphLAM: GraphLAM
interaction_nets:
    GraphLAM.InteractionNetWrapper.InteractionNet: InteractionNet
    GraphLAM.Sequential.InteractionNet: InteractionNet


In [ ]:
import torch
from torch import nn
import pytorch_lightning as pl

def trace_lightning_model_io(model, input_size):
    """
    Traces the PyTorch Lightning model to get the names of input and output tensors 
    along with the module hierarchy.

    Args:
        model (pl.LightningModule): The PyTorch Lightning model to trace.
        input_size (tuple): The size of the input tensor (e.g., (1, 3, 224, 224) for an image model).

    Returns:
        dict: A dictionary where keys are module names and values are tuples of (input_tensor_names, output_tensor_names).
    """
    def register_hooks(module, module_name, io_dict):
        def hook(module, input, output):
            input_names = [f"{module_name}_input_{i}" for i in range(len(input))]
            output_names = [f"{module_name}_output_{i}" for i in range(len(output) if isinstance(output, tuple) else 1)]
            io_dict[module_name] = (input_names, output_names)
        
        module.register_forward_hook(hook)

    io_dict = {}

    # Register hooks to all modules
    for name, module in model.named_modules():
        register_hooks(module, name, io_dict)

    # Create a dummy input tensor based on the input size
    dummy_input = torch.randn(input_size)

    # Perform a forward pass using PyTorch Lightning's predict method to trigger the hooks
    model.eval()
    with torch.no_grad():
        if hasattr(model, "predict_step"):
            model.predict_step(dummy_input)
        else:
            model(dummy_input)

    return io_dict

# Example usage:
# Assuming model is an instance of your PyTorch Lightning model class
# Replace (1, 3, 224, 224) with the appropriate input size for your model
hidden_dim = args.hidden_dim
hidden_layers = args.hidden_layers
mesh_aggr = args.mesh_aggr
lr = args.lr
dataset = args.dataset
output_std = args.output_std
loss = args.loss
step_length = args.step_length
n_example_pred = args.n_example_pred
graph = args.graph
processor_layers = args.processor_layers
model = GraphLAM(hidden_dim, hidden_layers, mesh_aggr,lr, dataset, output_std, loss, step_length, n_example_pred,graph,processor_layers)

model_io = trace_lightning_model_io(model, (1, 3, 224, 224))

# Print the module I/O information
for module_name, (input_names, output_names) in model_io.items():
    print(f"Module: {module_name}")
    print(f"  Input Tensors: {input_names}")
    print(f"  Output Tensors: {output_names}")

get_num_mesh
Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
Edges in subgraphs: m2m=51520, g2m=100656, m2g=255136
stage1
stage2


TypeError: BaseGraphModel.predict_step() missing 3 required positional arguments: 'prev_prev_state', 'batch_static_features', and 'forcing'

In [ ]:
import torch
from torch import nn
import pytorch_lightning as pl

def trace_lightning_model_io_with_loader(model, dataloader):
    """
    Traces the PyTorch Lightning model to get the names of input and output tensors 
    along with the module hierarchy using a provided DataLoader.

    Args:
        model (pl.LightningModule): The PyTorch Lightning model to trace.
        dataloader (torch.utils.data.DataLoader): The DataLoader to supply inputs to the model.

    Returns:
        dict: A dictionary where keys are module names and values are tuples of (input_tensor_names, output_tensor_names).
    """
    def register_hooks(module, module_name, io_dict):
        def hook(module, input, output):
            input_names = [f"{module_name}_input_{i}" for i in range(len(input))]
            output_names = [f"{module_name}_output_{i}" for i in range(len(output) if isinstance(output, tuple) else 1)]
            io_dict[module_name] = (input_names, output_names)
        
        module.register_forward_hook(hook)

    io_dict = {}

    # Register hooks to all modules
    for name, module in model.named_modules():
        register_hooks(module, name, io_dict)

    # Perform a forward pass using the dataloader to trigger the hooks
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            model.common_step(batch)  # Assuming common_step is used in predict_step or similar
            break  # Only need one batch to trace

    return io_dict

# Example usage:
# Assuming model is an instance of your PyTorch Lightning model class
# And dataloader is your DataLoader providing inputs

# Create an instance of your model and dataloader
# model = GraphLAM(...)  # Initialize with appropriate arguments
# eval_loader = torch.utils.data.DataLoader(...)  # Set up your DataLoader

model_io = trace_lightning_model_io_with_loader(model, eval_loader)

# Print the module I/O information
for module_name, (input_names, output_names) in model_io.items():
    print(f"Module: {module_name}")
    print(f"  Input Tensors: {input_names}")
    print(f"  Output Tensors: {output_names}")

NameError: name 'eval_loader' is not defined